In [87]:
import warnings
warnings.filterwarnings("ignore")
import os
os.chdir("/Users/mammoc/Desktop/Python")
import pandas as pd
airport = pd.read_csv("airport_original1.csv", sep = ';', header = 0)
#airport

In [88]:
# Dropping rows where 'Destination' is 4
airport = airport[airport['Destination'] != 4]
airport = airport.dropna(subset=['Destination'])

In [89]:
#FILL UP MISSING VALUES
#Age has 1 missing value (ID 214) => Replace by average age of 40
airport.loc[airport['ID'] == 214, 'Age'] = 40

#Gender has 3 missing values (ID 179, 199, 259)
# => Fill up by most frequency category - female - 2.0
airport.loc[airport['ID'].isin([179, 199, 259]), 'Gender'] = 2.0


#SeatClass has 3 missing value 
#Fill up by most frequency category = 1.0
airport.loc[airport['ID'].isin([245, 255, 450, 475]), 'SeatClass'] = 1.0

#AccessTime
AccessTime_by_ProvinceResidence = airport.groupby('ProvinceResidence')['AccessTime'].mean()
airport['AccessTime'] = airport.apply(
    lambda row: AccessTime_by_ProvinceResidence[row['ProvinceResidence']] if pd.isnull(row['AccessTime']) 
    else row['AccessTime'],axis=1)
#airport.loc[airport['ID'] == 102, 'AccessTime']
#AccessTime_by_ProvinceResidence


# Recategorize 'Airline' into binary categories: 'Legacy' and 'Other'
airport['Airline[Legacy]'] = airport['Airline'].isin([1, 2]).astype(int)

Airfare_by_Airline_Destination = airport.groupby(['Airline[Legacy]','Destination'])['Airfare'].mean()

#Fill the missing values in 'Airfare' with the average values calculated above
airport['Airfare'] = airport.apply(
    lambda row: Airfare_by_Airline_Destination[row['Airline[Legacy]'], row['Destination']] if pd.isnull(row['Airfare']) 
    else row['Airfare'], axis=1)


In [90]:
X = airport[['Age',
             'Gender',
             'TripPurpose',
             'Nationality',
             'TripDuration',
             'FlyingCompanion',
             'ProvinceResidence',
             'GroupTravel',
             'Destination',
             'DepartureTime',            
             'AccessTime',
             'NoTransport',
             'NoTripsLastYear',
             'SeatClass',
             'ModeTransport',
             'Airfare'
             ]]
#Seoul,Incheon and other, Incheon,Kyungki-do and others
y = airport['Airport']
random_state=109

In [138]:
#Standardize
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

mlp = MLPClassifier(hidden_layer_sizes=(10),activation='relu', max_iter=1000) #10: 81
mlp.fit(X, y)

MLPClassifier(hidden_layer_sizes=10, max_iter=1000)

In [139]:
from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation
scores = cross_val_score(mlp, X, y, cv=10)

# Print the accuracy for each fold
print("Accuracy for each fold: ", scores)

# Print the mean accuracy and standard deviation
print("Mean accuracy: {:.2f}".format(scores.mean()))


Accuracy for each fold:  [0.73913043 0.73913043 0.76086957 0.7826087  0.89130435 0.84782609
 0.84782609 0.80434783 0.84782609 0.86666667]
Mean accuracy: 0.81


In [126]:
airport['Airline[Legacy]'] = (airport['Airline'].isin([1,2])).astype(int)
X1 = airport[['Age',
             'Gender',
             'TripPurpose',
             'Occupation',
             'Nationality',
             'TripDuration',
             'FlyingCompanion',
             'ProvinceResidence',
             'GroupTravel',
             'Destination',
             'DepartureTime',            
             'AccessTime',
             'NoTransport',
             'NoTripsLastYear',
             'SeatClass',
             'ModeTransport',
             'Airfare'
             ]]
#Seoul,Incheon and other, Incheon,Kyungki-do and others
y1 = airport['Airline[Legacy]']
random_state=109

#Standardize
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
scaler = StandardScaler()
scaler.fit(X1)
X = scaler.transform(X1)

mlp = MLPClassifier(hidden_layer_sizes=(15),activation='logistic', max_iter=1000) #15: 67 or 71
mlp.fit(X1, y1)

MLPClassifier(activation='logistic', hidden_layer_sizes=15, max_iter=1000)

In [127]:
from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation
scores = cross_val_score(mlp, X1, y1, cv=10)

# Print the accuracy for each fold
print("Accuracy for each fold: ", scores)

# Print the mean accuracy and standard deviation
print("Mean accuracy: {:.2f}".format(scores.mean()))

Accuracy for each fold:  [0.73913043 0.84782609 0.69565217 0.63043478 0.67391304 0.63043478
 0.7173913  0.67391304 0.76086957 0.68888889]
Mean accuracy: 0.71
